In [5]:
import requests
import json
import pandas as pd
import feedparser as fp
import newspaper as np

def mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories):
    # Liste des nouveaux articles
    new_articles = []

    # Parcourir les flux RSS
    for source_url in flux_rss_a_lire:
        # Déterminer la catégorie du flux
        category = categories[source_url]

        data = fp.parse(source_url)
        
        # Parcourir les articles du flux RSS
        for item in data.entries:
            # Clés renommées pour correspondre à la base de données existante
            article = {
                "title": item.title,
                "date": item.published,  # Renommé de 'published' à 'date'
                "url": item.link,  # Renommé de 'link' à 'url'
                "content": item.description if "description" in item else "",
                "image_link": item.enclosures[0].url if "enclosures" in item and item.enclosures[0].type == "image/jpeg" else "",
                "category": category,  # Ajouté la catégorie
            }
            
            # Vérifier si l'article existe déjà dans la base de données
            if article["url"] not in database:
                article_obj = np.Article(article["url"])
                article_obj.download()
                article_obj.parse()
                
                # Mettre à jour les informations de l'article
                article["content"] += "\n\n" + article_obj.text  # Ajouter le contenu téléchargé
                article["author"] = article_obj.authors  # Ajouté la liste des auteurs
                
                # Ajouter l'article à la liste des nouveaux articles
                new_articles.append((article["url"], article))

    # Mettre à jour la base de données existante avec les nouveaux articles
    for url, article in new_articles:
        database[url] = article

    return database


# Lien vers le fichier tvinfo-sources.json
tvinfo_sources_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json"

# Charger la liste des flux RSS depuis le lien tvinfo-sources.json
response_sources = requests.get(tvinfo_sources_url)
sources = json.loads(response_sources.text)

# Liste des flux RSS à parcourir
flux_rss_a_lire = list(sources.values())

# Usage de la fonction avec les catégories
categories = {
    "https://www.francetvinfo.fr/france.rss": "france",
    "https://www.francetvinfo.fr/monde/europe.rss": "europe",  # Corrigé ici
    "https://www.francetvinfo.fr/economie/entreprises.rss": "economie"
}


# Charger la base de données existante depuis le lien francetvinfo.json
francetvinfo_url = "https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json"
response_database = requests.get(francetvinfo_url)
database = json.loads(response_database.text)

# Mettre à jour la base de données avec les nouveaux articles
database_mise_a_jour = mettre_a_jour_base_de_donnees(flux_rss_a_lire, database, categories)

# Transformer la base de données mise à jour en DataFrame
df_database_mise_a_jour = pd.DataFrame(database_mise_a_jour.values())

# Ajouter les URLs comme colonne dans le DataFrame
df_database_mise_a_jour['url'] = list(database_mise_a_jour.keys())

# Afficher les premières lignes pour vérifier
df_database_mise_a_jour





,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,NaN,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,NaN,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,NaN,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,NaN,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,NaN,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2152,Leasing social pour les voitures électriques :...,"Mon, 12 Feb 2024 12:54:42 +0100",[Emmanuel Cugny],economie,"Le ministre de l’Industrie et de l’énergie, Ro...",https://www.francetvinfo.fr/pictures/ccdYzaC2l...,NaN,https://www.francetvinfo.fr/replay-radio/le-de...
2153,Comment féminiser les métiers de terrain dans ...,"Sun, 11 Feb 2024 08:47:43 +0100",[Sarah Lemoine],economie,La part des femmes a légèrement progressé dans...,https://www.francetvinfo.fr/pictures/-GnreVvJ2...,NaN,https://www.francetvinfo.fr/replay-radio/c-est...
2154,Entreprises : la crise de l’immobilier pèse su...,"Fri, 09 Feb 2024 09:14:50 +0100",[Emmanuel Cugny],economie,"Les Français ont moins acheté de lave-linges, ...",https://www.francetvinfo.fr/pictures/AnPZT-kFs...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2155,"Semaine de quatre jours : ""Il va falloir ident...","Thu, 08 Feb 2024 20:19:53 +0100","[Emmanuel Cugny, Marie Bouny]",economie,"Marie Bouny, docteure en droit et associée au ...",,NaN,https://www.francetvinfo.fr/replay-radio/l-int...


# Mettre à jour la base de données de Git

In [6]:

# Fonction pour récupérer la base de données actuelle depuis GitHub
def telecharger_base_donnees_github(url):
    response = requests.get(url)
    data = response.text.splitlines()  # Séparer chaque ligne du fichier JSON
    df = pd.DataFrame([pd.read_json(line, typ='series') for line in data])
    return df

# Fonction pour concaténer l'ancienne base de données avec les nouveaux articles
def concatener_bases_donnees(df_ancienne, df_nouveaux_articles):
    # Filtrer les nouveaux articles pour ne garder que ceux qui n'existent pas déjà dans l'ancienne base de données
    nouveaux_articles_uniques = df_nouveaux_articles[~df_nouveaux_articles['url'].isin(df_ancienne['url'])]
    # Concaténer les deux DataFrames
    df_mise_a_jour = pd.concat([df_ancienne, nouveaux_articles_uniques], ignore_index=True)
    return df_mise_a_jour

# URL de la base de données sur GitHub
url_github = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'

# Charger la base de données actuelle
df_ancienne = telecharger_base_donnees_github(url_github)

# Concaténer l'ancienne base de données avec les nouveaux articles
df_mise_a_jour = concatener_bases_donnees(df_ancienne, df_database_mise_a_jour)

# Sauvegarder la base de données mise à jour localement
df_mise_a_jour.to_json("Data/francetvinfo_maj.json", orient="records", lines=True, force_ascii=False)
df_mise_a_jour.to_csv("Data/francetvinfo_maj.csv", index=False)


In [7]:
df_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2266,"Exportations en baisse, inflation importante.....","Thu, 22 Feb 2024 07:56:51 +0100",[Emmanuel Cugny],europe,La croissance économique ne sera pas au rendez...,https://www.francetvinfo.fr/pictures/oBS1ZqjVg...,NaN,https://www.francetvinfo.fr/replay-radio/le-br...
2267,"TEMOIGNAGE. ""Je ne parle à personne de cet arg...","Thu, 22 Feb 2024 07:45:32 +0100","[Vanessa Descouraux, Gilles Gallinaro, Olena, ...",europe,Une partie de la société ukrainienne critique ...,https://www.francetvinfo.fr/pictures/twN9UlUaw...,NaN,https://www.francetvinfo.fr/monde/europe/manif...
2268,Belgique : des ossements de mammouths découver...,"Wed, 21 Feb 2024 22:47:04 +0100",[],europe,La découverte est rarissime et incroyable. Des...,,NaN,https://www.francetvinfo.fr/monde/europe/belgi...
2269,Dette publique : alors que l'étau se resserre ...,"Wed, 21 Feb 2024 17:31:15 +0100","[Bruno Duvic, Mathieu De Taillac, Angélique]",europe,Le temps des restrictions budgétaires est-il r...,https://www.francetvinfo.fr/pictures/yx8pz0P9A...,NaN,https://www.francetvinfo.fr/replay-radio/le-cl...


In [8]:
import pandas as pd
# URL du fichier mise à jour chaque jour JSON sur GitHub
url = 'https://raw.githubusercontent.com/diakitegaoussou4996/ENSAI-PROJET-WEBMING/main/Data/francetvinfo_maj.json'
# Charger les données dans un DataFrame
francetvinfo_mise_a_jour = pd.read_json(url, lines=True)
# Afficher les premières lignes du DataFrame
francetvinfo_mise_a_jour

,title,date,author,category,content,image_link,image_file,url
0,"Violences contre les maires : dans l'Eure, des...",2020-01-19T16:31:16,[],france,Beaucoup de maires et de secrétaires de mairie...,https://www.francetvinfo.fr/image/75rt1bxrp-70...,None,https://www.francetvinfo.fr/sante/prevention/e...
1,VIDEO. Sur le dos d'un aigle pêcheur au-dessus...,2020-01-19T16:30:02,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1br51-6d...,None,https://www.francetvinfo.fr/france/video-sur-l...
2,"VIDEO. Dans le massif du Mont-Blanc, la hausse...",2020-01-19T16:29:55,[],france,"#AlertePollution Rivières ou sols contaminés, ...",https://www.francetvinfo.fr/image/75rt1bbzf-01...,None,https://www.francetvinfo.fr/societe/video-mass...
3,"""13h15 le dimanche"". Le Mont-Blanc, un mythe é...",2020-01-19T16:29:48,[],france,Point culminant de la chaîne des Alpes avec un...,https://www.francetvinfo.fr/image/75rszkkvp-24...,None,https://www.francetvinfo.fr/replay-magazine/fr...
4,Loi de bioéthique : les opposants à la PMA man...,2020-01-19T16:08:10,[],france,"Des associations catholiques, des jeunes, des ...",https://www.francetvinfo.fr/image/75rt1bbs2-3f...,None,https://www.francetvinfo.fr/societe/loi-de-bio...
...,...,...,...,...,...,...,...,...
2240,"Calvados : à Ouistreham, les terrains municipa...","Wed, 21 Feb 2024 15:49:03 +0100",[],france,"La ville de Ouistreham, en Normandie, propose ...",,None,https://www.francetvinfo.fr/france/normandie/c...
2241,Fraude : de plus en plus de tricheries dans le...,"Wed, 21 Feb 2024 15:45:18 +0100",[],france,Plusieurs centres d'examen du code de la route...,,None,https://www.francetvinfo.fr/economie/fraude/fr...
2242,Allergies aux pollens : pourquoi le Finistère ...,"Wed, 21 Feb 2024 14:32:47 +0100",[Valentine],france,Alors que le risque maximal pour les allergies...,https://www.francetvinfo.fr/pictures/bqMm2iD4H...,None,https://www.francetvinfo.fr/sante/allergies-au...
2243,"""Au lieu d'aider cette femme inconsciente sur ...","Wed, 21 Feb 2024 14:31:16 +0100","[Phéline Leloir-Duault, Maxime Leclerc, Marian...",france,"Témoin de la scène, qu'il a raconté sur X, Max...",https://www.francetvinfo.fr/pictures/oKAKdR04d...,None,https://www.francetvinfo.fr/sante/psycho-bien-...
